In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model_name = "gpt-4o-mini"

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

# memory = SqliteSaver.from_conn_string(":memory:")
memory = SqliteSaver.from_conn_string("checkpoints_memory.db")

In [ ]:
# route  if it is necessary to retrieve new documents to answer or it is a follow up question about the conversation. route to retriever or to generate answer

# - route generate
    # generate answer based in the conversation


# - route retrieve
    # create a new query to use to retireve docs
    # retrieve docs
    # generate answer using the docs retrieved

In [5]:
class AgentState(TypedDict):
    user_input: str
    should_retrieve_new_docs: bool
    new_query: str
    docs_retrieved: str
    critique: str

In [8]:
state = AgentState()
state['user_input'] = 'como faço para instalar o pje?'

In [19]:
from langchain_core.pydantic_v1 import BaseModel, Field

class RouteInput(BaseModel):
    """Binary score to asses if the user input could be answered using the current conversation memory history or it is needed to retrieve documents from a vector store."""

    should_retrieve_new_docs: bool = Field(
        description="Do you need additional documents to answer the user input?, True or False"
    )

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def route_input_user_node(state: AgentState):
    llm = ChatOpenAI(model=model_name, temperature=0)
    llm_RouteInput = llm.with_structured_output(RouteInput)

    # Prompt
    PROMPT_ROUTE_INPUT = """
    You are an expert at assessing if a user input could be answered using the current conversation memory history \
        or it is needed to retrieve documents from a vector store. 
    If the input is asking a follow up question about the current conversation memory history, you should output 'no'.
    If the input is asking a question that need information that couldn't be found in the current conversation memory history, you should output 'yes'.
    """

    llm_RouteInput_response = llm_RouteInput.invoke([
            SystemMessage(content=PROMPT_ROUTE_INPUT),
            HumanMessage(content=state['user_input'])
        ])

    return { 'should_retrieve_new_docs': llm_RouteInput_response.should_retrieve_new_docs }

In [25]:
returned = route_input_user_node(state)
returned

{'should_retrieve_new_docs': True}

In [26]:
state['should_retrieve_new_docs'] = returned['should_retrieve_new_docs']

In [27]:
state

{'user_input': 'como faço para instalar o pje?',
 'should_retrieve_new_docs': True}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

def generate_answer_conversation_node(state: AgentState):
    llm = ChatOpenAI(model=model_name, temperature=0)

    # Prompt
    PROMPT_SYSTEM = """
    You are a helpfull assistant 

    """

    llm_RouteInput_response = llm_RouteInput.invoke([
            SystemMessage(content=PROMPT_SYSTEM),
            HumanMessage(content=state['user_input'])
        ])

    return { 'should_retrieve_new_docs': llm_RouteInput_response.should_retrieve_new_docs }